# Basic Neural Robot Control - Training

The objective of this experiment is to control the Cozmo robot using a neural network, attempting to replicate the behaviour patterns of a human controller. 

The approach used here is to use an LSTM RNN to predict the next action taken by the human from a fixed-length sequence of environment observations and prevously taken actions. 

In the current version, the following environment observations are taken into consideration:
- The inferred position and angle of the robot (as provided by the Cozmo API)
- The positions of each of the three light-cubes (if visible)

The following actions are available:
- Drive forward
- Drive back
- Turn left
- Turn right
- Lift up 
- Lift down
- Head up
- Head down

The vector of observations is concatenated together with the vector of actions (represented as 1.0 / 0.0) to produce a single time-step to the RNN. The output of the network is the actions.

The training approach involves using a sliding window over the training data set with two hyperparameters:
- *horizon* - the number of steps (recording snapshots) to be examined by the network in a single step
- *step size*  - (usually, a smaller value) the number of observations the sliding window is moved by for each training example

The network will attempt to predict the actions in the first snapshot outside of the sliding window. 

Directions for future experiments (TODO):
- Adding the rotation quaternions of each cube
- Adding a scene embedding produced from the raw camera footage
- Adding metrics and a test set
- Adding dropout

In [105]:
import pandas as pd
import tensorflow as tf
import numpy as np
from glob import glob
K = tf.keras
L = K.layers
from ipywidgets import interact_manual
import ipywidgets as widgets
import matplotlib.pyplot as plt

In [7]:
def read_recordings(pattern):
    return pd.concat(pd.read_hdf(file_name, 'robot') for file_name in glob(pattern)).reset_index(drop=True)

In [8]:
control_data = read_recordings('robot-*.h5')
control_data

,robot_position_x,robot_position_y,robot_angle,cube_1_visible,cube_1_x,cube_1_y,cube_2_visible,cube_2_x,cube_2_y,cube_3_visible,cube_3_x,cube_3_y,action_drive_forwards,action_drive_back,action_turn_left,action_turn_right,action_lift_up,action_lift_down,action_head_up,action_head_down
0,43.889851,-18.560421,-0.440358,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
1,43.889851,-18.560421,-0.440384,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
2,43.889851,-18.560421,-0.440375,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
3,43.889851,-18.560421,-0.440375,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
4,43.889851,-18.560421,-0.440348,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
5,43.889851,-18.560421,-0.440348,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
6,43.889851,-18.560421,-0.440311,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,False,0,0,False,0,False,0,False
7,43.889851,-18.560421,-0.440288,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,True,0,0,False,0,False,0,False
8,43.889851,-18.560421,-0.440288,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,True,0,0,False,0,False,0,False
9,43.889851,-18.560421,-0.440298,False,0.0,0.0,False,0.0,0.0,False,0.0,0.0,True,0,0,False,0,False,0,False


In [9]:
control_data = control_data.astype(np.float64)

In [10]:
def get_actions(data):
    return [col for col in data.columns if col.startswith('action_')]

In [55]:
def get_model(data_sample, horizon):
    control_vector_size = len(data_sample.columns)
    actions = get_actions(data_sample)
    model = K.Sequential()
    model.add(L.LSTM(32, return_sequences=True, stateful=False,
                     input_shape=(horizon, control_vector_size)))
    model.add(L.LSTM(32, return_sequences=True, stateful=False))
    model.add(L.LSTM(32, return_sequences=False, stateful=False))
    model.add(L.Dense(len(actions), activation='hard_sigmoid'))
    return model

In [146]:
def get_control_sample(control_data_matrix, x, horizon):
    sample = control_data_matrix[x:x+horizon]
    sample = np.pad(
        sample, 
        pad_width=((0, (horizon - len(sample))), (0, 0)),
        mode='edge' if len(sample) else 'constant') 
    return sample

def get_batch(control_data_matrix, horizon, step_size, x, batch_size):
    batch_range = range(x, x+step_size*batch_size, step_size)
    batch_x = np.stack(
        get_control_sample(control_data_matrix, x1, horizon)
        for x1 in batch_range
    )
    batch_y = np.stack(
        np.squeeze(get_control_sample(control_data_matrix, x + horizon, 1)[:, -8:])
        for x1 in batch_range
    )
    return batch_x, batch_y

def get_data_generator(control_data, horizon, step_size, batch_size):
    control_data_matrix = control_data.astype(np.float64).as_matrix()
    return (
        get_batch(control_data_matrix, horizon, step_size, x, batch_size)
        for x in range(0, len(control_data), step_size*batch_size))

In [150]:
horizon = 32
step_size = 5
model = get_model(control_data, horizon=horizon)
actions = get_actions(control_data)
model.compile(loss='mean_squared_error', optimizer='rmsprop')
batch_size = 20

In [166]:
epochs = 10
for _ in range(epochs):
    generator = get_data_generator(control_data, horizon=horizon, step_size=step_size, batch_size=batch_size)
    model.fit_generator(generator, steps_per_epoch=len(control_data)/step_size/batch_size)

Epoch 1/1
75/74 [==============================] - 8s 110ms/step - loss: 0.0441
Epoch 1/1
75/74 [==============================] - 8s 112ms/step - loss: 0.0411
Epoch 1/1
75/74 [==============================] - 8s 108ms/step - loss: 0.0423
Epoch 1/1
75/74 [==============================] - 8s 110ms/step - loss: 0.0421
Epoch 1/1
75/74 [==============================] - 8s 109ms/step - loss: 0.0422
Epoch 1/1
75/74 [==============================] - 9s 114ms/step - loss: 0.0410
Epoch 1/1
75/74 [==============================] - 8s 113ms/step - loss: 0.0394
Epoch 1/1
75/74 [==============================] - 8s 113ms/step - loss: 0.0393
Epoch 1/1
75/74 [==============================] - 9s 119ms/step - loss: 0.0389
Epoch 1/1
75/74 [==============================] - 9s 117ms/step - loss: 0.0379


In [160]:
%matplotlib inline
min_robot_x, max_robot_x = control_data.robot_position_x.min(), control_data.robot_position_x.max()
min_robot_y, max_robot_y = control_data.robot_position_y.min(), control_data.robot_position_y.max()
arrow_length = 100
arrow_width = 2
padding = 20
actions = [column.replace('action_', '') for column in control_data.columns
           if column.startswith('action_')]
control_data_matrix = control_data.astype(np.float64).as_matrix()

@interact_manual(x=widgets.IntText(min=0, max=len(control_data), continuous_update=False))
def plot(x):
    plt.figure(figsize=(12, 8))
    sample = control_data.loc[x+horizon]
    prediction_sample = get_control_sample(control_data_matrix, x, horizon)
    predicted_actions = np.squeeze(model.predict(np.expand_dims(prediction_sample, axis=0)))
    plt.subplot(2, 1, 1)
    plt.xlim(min_robot_x - padding, max_robot_x + padding)
    plt.ylim(min_robot_y - padding, max_robot_y + padding)
    plt.plot(sample.robot_position_x, sample.robot_position_y, 'ro')
    plt.arrow(sample.robot_position_x, sample.robot_position_y, 
              arrow_length * np.cos(sample.robot_angle),
              arrow_length * np.sin(sample.robot_angle),
              width=arrow_width)
    plt.subplot(2, 1, 2)
    actions_range = range(len(actions))    
    plt.bar(actions_range, predicted_actions)
    plt.xticks(actions_range, actions)
    plt.xlim(-1, len(actions))
    plt.ylim(0, 1.0)
    for idx, action in enumerate(actions):
        if sample[f'action_{action}']:
            plt.plot(idx, predicted_actions[idx], 'ro')
    
    for cube_idx in range(1, 4):
        if sample[f'cube_{cube_idx}_visible']:
            plt.plot(sample[f'cube_{cube_idx}_x'], sample[f'cube_{cube_idx}_y'], 'b+')
    

interactive(children=(IntText(value=0, description='x'), Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

In [167]:
model.save('basic_neural_control_lstm.h5')